# * VINSIGHT : Inflow
    Prepaid(GEO Channel) : DTAC รอพี่หมอ re-run ต้องไม่นับ Wholesales

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Transaction

In [2]:
''' Execute transaction '''


# Input parameter
v_start_date = 20250101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B0R00010001CS' --Total Inflow M1
        , 'TB0R00010001CS' --Total Inflow M1 : TRUE
        , 'DB0R00010001CS' --Total Inflow M1 : DTAC
        , 'B0R00010001CG' --Total Inflow M1 - GEO Channel
        , 'TB0R00010001CG' --Total Inflow M1 : TRUE - GEO Channel
        , 'DB0R00010001CG' --Total Inflow M1 : DTAC - GEO Channel
        
        , 'B1R000900CS' --Prepaid Inflow M1
        , 'TB1R000900CS' --Prepaid Inflow M1 : TMH
        , 'DB1R000900CS' --Prepaid Inflow M1 : DTAC
        , 'B1R000900CG' --Prepaid Inflow M1 - GEO Channel
        , 'TB1R000900CG' --Prepaid Inflow M1 : TMH - GEO Channel
        , 'DB1R000900CG' --Prepaid Inflow M1 : DTAC - GEO Channel
        
        , 'B2R010500CS' --Postpaid Inflow M1 B2C
        , 'TB2R010500CS' --Postpaid Inflow M1 B2C : TMH
        , 'DB2R010500CS' --Postpaid Inflow M1 B2C : DTAC
        , 'B2R010500CG' --Postpaid Inflow M1 B2C - GEO Channel
        , 'TB2R010500CG' --Postpaid Inflow M1 B2C : TMH - GEO Channel
        , 'DB2R010500CG' --Postpaid Inflow M1 B2C : DTAC - GEO Channel
        , 'B2R020500CS' --Postpaid Inflow M1 B2B
        , 'TB2R020500CS' --Postpaid Inflow M1 B2B : TMH
        , 'DB2R020500CS' --Postpaid Inflow M1 B2B : DTAC
        , 'B2R020500CG' --Postpaid Inflow M1 B2B - GEO Channel
        , 'DB2R020500CG' --Postpaid Inflow M1 B2B : DTAC - GEO Channel
        , 'TB2R020500CG' --Postpaid Inflow M1 B2B : TMH - GEO Channel
        
        , 'TB3R000601CS' --TOL Inflow M1 Connected : Consumer
        , 'TB3R000601CG' --TOL Inflow M1 Connected : Consumer - GEO Channel
        , 'TB3R000601D1CS' --TOL Inflow M1 Connected : Consumer (Install Location)
        , 'TB3R000601D1CG' --TOL Inflow M1 Connected : Consumer - GEO Channel (Install Location)
        , 'TB3R000602CS' --TOL Inflow M1 Connected : DataService
        , 'TB3R000602CG' --TOL Inflow M1 Connected : DataService - GEO Channel
        , 'TB3R000602D1CS' --TOL Inflow M1 Connected : DataService (Install Location)
        , 'TB3R000602D1CG' --TOL Inflow M1 Connected : DataService - GEO Channel (Install Location)

        , 'TB4R001700CS' --TVS Now Inflow M1
        , 'TB4R001700CG' --TVS Now Inflow M1 - GEO Channel
        , 'TB4R001004CS' --TVS CMDU Inflow M1
        , 'TB4R001004CG' --TVS CMDU Inflow M1 - GEO Channel
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    --AND TM_KEY_DAY IN (20240131, 20240229, 20240331, 20240430, 20240531, 20240630, 20240731, 20240831, 20240930, 20241031, 20241130, 20241231, 20250131, 20250228, 20250331) -->> Year 2024
    --AND TM_KEY_DAY IN (20250131, 20250228, 20250331, 20250430, 20250531) -->> Year 2025
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\nDataFrame: {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
    
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20250101

Data as of 2025-06-17, 13:02:34

DataFrame: 6012 rows, 16 columns


In [3]:
''' Automate Currently Period '''

curr_yr = chk_src_df['TM_KEY_YR'].max()
prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_qtr = chk_src_df['TM_KEY_QTR'].max()
prev_qtr = chk_src_df['TM_KEY_QTR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = chk_src_df['TM_KEY_MTH'].drop_duplicates().sort_values().shift().max().astype(int)

curr_wk = chk_src_df['TM_KEY_WK'].max()
prev_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift().max().astype(int)
last_3_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift(3).max().astype(int)

# curr_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].max().astype(int)
# prev_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].drop_duplicates().shift().max().astype(int)

C:\Users\Narut4\AppData\Local\Temp\ipykernel_16636\1881320084.py:4: RuntimeWarning: invalid value encountered in cast
  prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)


## Overview(All) by Period

In [4]:
''' Inflow M1(All Channel) Yearly '''

v_metric_list = [
    'B0R00010001CS' #Total Inflow M1
    , 'TB0R00010001CS' #Total Inflow M1 : TRUE
    , 'DB0R00010001CS' #Total Inflow M1 : DTAC

    , 'B1R000900CS' #Prepaid Inflow M1
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC

    , 'B2R010500CS' #Postpaid Inflow M1 B2C
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    # , 'B2R020500CS' #Postpaid Inflow M1 B2B
    # , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    # , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC

    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    # , 'TB3R000601D1CS' #TOL Inflow M1 Connected : Consumer (Install Location)
    # , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    # , 'TB3R000602D1CS' #TOL Inflow M1 Connected : DataService (Install Location)

    , 'TB4R001700CS' #TVS Now Inflow M1
    # , 'TB4R001004CS' #TVS CMDU Inflow M1
    ]

m1_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
m1_yearly_df = m1_yearly_df.loc[m1_yearly_df['METRIC_CD'].isin(v_metric_list)]

m1_yearly_df['TOTAL'] = np.where(m1_yearly_df['METRIC_CD']=='B0R00010001CS', m1_yearly_df['P'], 0)
m1_yearly_df['TOTAL_T'] = np.where(m1_yearly_df['METRIC_CD']=='TB0R00010001CS', m1_yearly_df['P'], 0)
m1_yearly_df['TOTAL_D'] = np.where(m1_yearly_df['METRIC_CD']=='DB0R00010001CS', m1_yearly_df['P'], 0)
m1_yearly_df['PRE'] = np.where(m1_yearly_df['METRIC_CD']=='B1R000900CS', m1_yearly_df['P'], 0)
m1_yearly_df['PRE_T'] = np.where(m1_yearly_df['METRIC_CD']=='TB1R000900CS', m1_yearly_df['P'], 0)
m1_yearly_df['PRE_D'] = np.where(m1_yearly_df['METRIC_CD']=='DB1R000900CS', m1_yearly_df['P'], 0)
m1_yearly_df['POST_B2C'] = np.where(m1_yearly_df['METRIC_CD']=='B2R010500CS', m1_yearly_df['P'], 0)
m1_yearly_df['POST_B2C_T'] = np.where(m1_yearly_df['METRIC_CD']=='TB2R010500CS', m1_yearly_df['P'], 0)
m1_yearly_df['POST_B2C_D'] = np.where(m1_yearly_df['METRIC_CD']=='DB2R010500CS', m1_yearly_df['P'], 0)
# m1_yearly_df['POST_B2B'] = np.where(m1_yearly_df['METRIC_CD']=='B2R020500CS', m1_yearly_df['P'], 0)
# m1_yearly_df['POST_B2B_T'] = np.where(m1_yearly_df['METRIC_CD']=='TB2R020500CS', m1_yearly_df['P'], 0)
# m1_yearly_df['POST_B2B_D'] = np.where(m1_yearly_df['METRIC_CD']=='DB2R020500CS', m1_yearly_df['P'], 0)
m1_yearly_df['TOL_CONN'] = np.where(m1_yearly_df['METRIC_CD']=='TB3R000601CS', m1_yearly_df['P'], 0)
# m1_yearly_df['TOL_CONN(Install)'] = np.where(m1_yearly_df['METRIC_CD']=='TB3R000601D1CS', m1_yearly_df['P'], 0)
# m1_yearly_df['TOL_DSV'] = np.where(m1_yearly_df['METRIC_CD']=='TB3R000602CS', m1_yearly_df['P'], 0)
# m1_yearly_df['TOL_DSV(Install)'] = np.where(m1_yearly_df['METRIC_CD']=='TB3R000602D1CS', m1_yearly_df['P'], 0)
m1_yearly_df['TVS_NOW'] = np.where(m1_yearly_df['METRIC_CD']=='TB4R001700CS', m1_yearly_df['P'], 0)
# m1_yearly_df['TVS_CMDU'] = np.where(m1_yearly_df['METRIC_CD']=='TB4R001004CS', m1_yearly_df['P'], 0)

# m1_yearly_df = m1_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_yearly_df['YoY'] = m1_yearly_df['TOTAL'] - m1_yearly_df['TOTAL'].shift()
# m1_yearly_df = m1_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
# m1_yearly_df = m1_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_yearly_df = m1_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_yearly_df['YoY'] = m1_yearly_df['TOTAL'] - m1_yearly_df['TOTAL'].shift()
m1_yearly_df = m1_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
m1_yearly_df = m1_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'YoY', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_yearly_df_display = m1_yearly_df.copy()
m1_yearly_df_display['ACTUAL_AS_OF'] = m1_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_yearly_df_display[col] = m1_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,YoY,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,2025,2025-06-17 00:53:59,20250615,0,"3,063,128,518","1,620,136,949","1,442,991,569","2,329,000,348","1,074,054,815","1,260,827,014","452,466,767","354,280,329","100,292,477","147,719,428","2,349,710"


In [5]:
''' Inflow M1(All Channel) Quarterly '''

v_metric_list = [
    'B0R00010001CS' #Total Inflow M1
    , 'TB0R00010001CS' #Total Inflow M1 : TRUE
    , 'DB0R00010001CS' #Total Inflow M1 : DTAC

    , 'B1R000900CS' #Prepaid Inflow M1
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC

    , 'B2R010500CS' #Postpaid Inflow M1 B2C
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    # , 'B2R020500CS' #Postpaid Inflow M1 B2B
    # , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    # , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC

    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    # , 'TB3R000601D1CS' #TOL Inflow M1 Connected : Consumer (Install Location)
    # , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    # , 'TB3R000602D1CS' #TOL Inflow M1 Connected : DataService (Install Location)

    , 'TB4R001700CS' #TVS Now Inflow M1
    # , 'TB4R001004CS' #TVS CMDU Inflow M1
    ]

m1_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
m1_quarterly_df = m1_quarterly_df.loc[m1_quarterly_df['METRIC_CD'].isin(v_metric_list)]

m1_quarterly_df['TOTAL'] = np.where(m1_quarterly_df['METRIC_CD']=='B0R00010001CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['TOTAL_T'] = np.where(m1_quarterly_df['METRIC_CD']=='TB0R00010001CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['TOTAL_D'] = np.where(m1_quarterly_df['METRIC_CD']=='DB0R00010001CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['PRE'] = np.where(m1_quarterly_df['METRIC_CD']=='B1R000900CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['PRE_T'] = np.where(m1_quarterly_df['METRIC_CD']=='TB1R000900CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['PRE_D'] = np.where(m1_quarterly_df['METRIC_CD']=='DB1R000900CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['POST_B2C'] = np.where(m1_quarterly_df['METRIC_CD']=='B2R010500CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['POST_B2C_T'] = np.where(m1_quarterly_df['METRIC_CD']=='TB2R010500CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['POST_B2C_D'] = np.where(m1_quarterly_df['METRIC_CD']=='DB2R010500CS', m1_quarterly_df['P'], 0)
# m1_quarterly_df['POST_B2B'] = np.where(m1_quarterly_df['METRIC_CD']=='B2R020500CS', m1_quarterly_df['P'], 0)
# m1_quarterly_df['POST_B2B_T'] = np.where(m1_quarterly_df['METRIC_CD']=='TB2R020500CS', m1_quarterly_df['P'], 0)
# m1_quarterly_df['POST_B2B_D'] = np.where(m1_quarterly_df['METRIC_CD']=='DB2R020500CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['TOL_CONN'] = np.where(m1_quarterly_df['METRIC_CD']=='TB3R000601CS', m1_quarterly_df['P'], 0)
# m1_quarterly_df['TOL_CONN(Install)'] = np.where(m1_quarterly_df['METRIC_CD']=='TB3R000601D1CS', m1_quarterly_df['P'], 0)
# m1_quarterly_df['TOL_DSV'] = np.where(m1_quarterly_df['METRIC_CD']=='TB3R000602CS', m1_quarterly_df['P'], 0)
# m1_quarterly_df['TOL_DSV(Install)'] = np.where(m1_quarterly_df['METRIC_CD']=='TB3R000602D1CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['TVS_NOW'] = np.where(m1_quarterly_df['METRIC_CD']=='TB4R001700CS', m1_quarterly_df['P'], 0)
# m1_quarterly_df['TVS_CMDU'] = np.where(m1_quarterly_df['METRIC_CD']=='TB4R001004CS', m1_quarterly_df['P'], 0)

# m1_quarterly_df = m1_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_quarterly_df['QoQ'] = m1_quarterly_df['TOTAL'] - m1_quarterly_df['TOTAL'].shift()
# m1_quarterly_df = m1_quarterly_df.fillna(0).sort_values(by=['TM_KEY_DAY']).reset_index()
# m1_quarterly_df = m1_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_quarterly_df = m1_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_quarterly_df['QoQ'] = m1_quarterly_df['TOTAL'] - m1_quarterly_df['TOTAL'].shift()
m1_quarterly_df = m1_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
m1_quarterly_df = m1_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'QoQ', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_quarterly_df_display = m1_quarterly_df.copy()
m1_quarterly_df_display['ACTUAL_AS_OF'] = m1_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_quarterly_df_display[col] = m1_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,QoQ,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,20251,2025-06-17 00:53:59,20250331,0,"1,778,654,197","918,417,080","860,237,117","1,379,397,681","619,695,416","759,702,265","249,140,023","195,429,182","53,710,841","79,484,928","1,120,189"
1,20252,2025-06-17 00:53:59,20250615,"-494,179,876","1,284,474,321","701,719,869","582,754,452","949,602,666","454,359,399","501,124,750","203,326,743","158,851,146","46,581,636","68,234,500","1,229,522"


In [6]:
''' Inflow M1(All Channel) Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
    'B0R00010001CS' #Total Inflow M1
    , 'TB0R00010001CS' #Total Inflow M1 : TRUE
    , 'DB0R00010001CS' #Total Inflow M1 : DTAC

    , 'B1R000900CS' #Prepaid Inflow M1
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC

    , 'B2R010500CS' #Postpaid Inflow M1 B2C
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    # , 'B2R020500CS' #Postpaid Inflow M1 B2B
    # , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    # , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC

    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    # , 'TB3R000601D1CS' #TOL Inflow M1 Connected : Consumer (Install Location)
    # , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    # , 'TB3R000602D1CS' #TOL Inflow M1 Connected : DataService (Install Location)

    , 'TB4R001700CS' #TVS Now Inflow M1
    # , 'TB4R001004CS' #TVS CMDU Inflow M1
    ]

m1_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr].copy()
m1_monthly_df = m1_monthly_df.loc[m1_monthly_df['METRIC_CD'].isin(v_metric_list)]

m1_monthly_df['TOTAL'] = np.where(m1_monthly_df['METRIC_CD']=='B0R00010001CS', m1_monthly_df['P'], 0)
m1_monthly_df['TOTAL_T'] = np.where(m1_monthly_df['METRIC_CD']=='TB0R00010001CS', m1_monthly_df['P'], 0)
m1_monthly_df['TOTAL_D'] = np.where(m1_monthly_df['METRIC_CD']=='DB0R00010001CS', m1_monthly_df['P'], 0)
m1_monthly_df['PRE'] = np.where(m1_monthly_df['METRIC_CD']=='B1R000900CS', m1_monthly_df['P'], 0)
m1_monthly_df['PRE_T'] = np.where(m1_monthly_df['METRIC_CD']=='TB1R000900CS', m1_monthly_df['P'], 0)
m1_monthly_df['PRE_D'] = np.where(m1_monthly_df['METRIC_CD']=='DB1R000900CS', m1_monthly_df['P'], 0)
m1_monthly_df['POST_B2C'] = np.where(m1_monthly_df['METRIC_CD']=='B2R010500CS', m1_monthly_df['P'], 0)
m1_monthly_df['POST_B2C_T'] = np.where(m1_monthly_df['METRIC_CD']=='TB2R010500CS', m1_monthly_df['P'], 0)
m1_monthly_df['POST_B2C_D'] = np.where(m1_monthly_df['METRIC_CD']=='DB2R010500CS', m1_monthly_df['P'], 0)
# m1_monthly_df['POST_B2B'] = np.where(m1_monthly_df['METRIC_CD']=='B2R020500CS', m1_monthly_df['P'], 0)
# m1_monthly_df['POST_B2B_T'] = np.where(m1_monthly_df['METRIC_CD']=='TB2R020500CS', m1_monthly_df['P'], 0)
# m1_monthly_df['POST_B2B_D'] = np.where(m1_monthly_df['METRIC_CD']=='DB2R020500CS', m1_monthly_df['P'], 0)
m1_monthly_df['TOL_CONN'] = np.where(m1_monthly_df['METRIC_CD']=='TB3R000601CS', m1_monthly_df['P'], 0)
# m1_monthly_df['TOL_CONN(Install)'] = np.where(m1_monthly_df['METRIC_CD']=='TB3R000601D1CS', m1_monthly_df['P'], 0)
# m1_monthly_df['TOL_DSV'] = np.where(m1_monthly_df['METRIC_CD']=='TB3R000602CS', m1_monthly_df['P'], 0)
# m1_monthly_df['TOL_DSV(Install)'] = np.where(m1_monthly_df['METRIC_CD']=='TB3R000602D1CS', m1_monthly_df['P'], 0)
m1_monthly_df['TVS_NOW'] = np.where(m1_monthly_df['METRIC_CD']=='TB4R001700CS', m1_monthly_df['P'], 0)
# m1_monthly_df['TVS_CMDU'] = np.where(m1_monthly_df['METRIC_CD']=='TB4R001004CS', m1_monthly_df['P'], 0)

# m1_monthly_df = m1_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_monthly_df['MoM'] = m1_monthly_df['TOTAL'] - m1_monthly_df['TOTAL'].shift()
# m1_monthly_df = m1_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
# m1_monthly_df = m1_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_monthly_df = m1_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_monthly_df['MoM'] = m1_monthly_df['TOTAL'] - m1_monthly_df['TOTAL'].shift()
m1_monthly_df = m1_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
m1_monthly_df = m1_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'MoM', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_monthly_df_display = m1_monthly_df.copy()
m1_monthly_df_display['ACTUAL_AS_OF'] = m1_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_monthly_df_display[col] = m1_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,MoM,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,202501,2025-06-17 00:53:59,20250131,0,"639,956,235","319,922,636","320,033,599","508,791,417","225,749,189","283,042,228","79,460,514","61,334,089","18,126,426","25,646,982","382,335"
1,202502,2025-06-17 00:53:59,20250228,"-64,775,833","575,180,401","288,071,249","287,109,152","442,103,679","192,368,192","249,735,487","81,132,607","63,185,463","17,947,144","25,046,745","267,183"
2,202503,2025-06-17 00:53:59,20250331,"-11,662,840","563,517,561","310,423,195","253,094,366","428,502,585","201,578,035","226,924,550","88,546,902","70,909,630","17,637,272","28,791,201","470,671"
3,202504,2025-06-17 00:53:59,20250430,"-29,481,414","534,036,147","295,795,804","238,240,343","395,842,378","191,822,637","204,019,741","87,789,592","69,118,915","18,670,677","26,236,110","441,580"
4,202505,2025-06-17 00:53:59,20250531,"-1,409,852","532,626,295","287,234,164","245,392,131","400,164,161","190,514,414","209,649,746","79,167,109","59,723,743","19,443,366","28,260,212","556,972"
5,202506,2025-06-17 00:53:59,20250615,"-314,814,416","217,811,879","118,689,901","99,121,977","153,596,128","72,022,348","87,455,262","36,370,042","30,008,488","8,467,593","13,738,178","230,970"


In [7]:
''' Inflow M1(All Channel) Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'B0R00010001CS' #Total Inflow M1
    , 'TB0R00010001CS' #Total Inflow M1 : TRUE
    , 'DB0R00010001CS' #Total Inflow M1 : DTAC

    , 'B1R000900CS' #Prepaid Inflow M1
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC

    , 'B2R010500CS' #Postpaid Inflow M1 B2C
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    # , 'B2R020500CS' #Postpaid Inflow M1 B2B
    # , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    # , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC

    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    # , 'TB3R000601D1CS' #TOL Inflow M1 Connected : Consumer (Install Location)
    # , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    # , 'TB3R000602D1CS' #TOL Inflow M1 Connected : DataService (Install Location)

    , 'TB4R001700CS' #TVS Now Inflow M1
    # , 'TB4R001004CS' #TVS CMDU Inflow M1
    ]

m1_weekly_df = chk_src_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
m1_weekly_df = m1_weekly_df.loc[m1_weekly_df['METRIC_CD'].isin(v_metric_list)]

m1_weekly_df['TOTAL'] = np.where(m1_weekly_df['METRIC_CD']=='B0R00010001CS', m1_weekly_df['P'], 0)
m1_weekly_df['TOTAL_T'] = np.where(m1_weekly_df['METRIC_CD']=='TB0R00010001CS', m1_weekly_df['P'], 0)
m1_weekly_df['TOTAL_D'] = np.where(m1_weekly_df['METRIC_CD']=='DB0R00010001CS', m1_weekly_df['P'], 0)
m1_weekly_df['PRE'] = np.where(m1_weekly_df['METRIC_CD']=='B1R000900CS', m1_weekly_df['P'], 0)
m1_weekly_df['PRE_T'] = np.where(m1_weekly_df['METRIC_CD']=='TB1R000900CS', m1_weekly_df['P'], 0)
m1_weekly_df['PRE_D'] = np.where(m1_weekly_df['METRIC_CD']=='DB1R000900CS', m1_weekly_df['P'], 0)
m1_weekly_df['POST_B2C'] = np.where(m1_weekly_df['METRIC_CD']=='B2R010500CS', m1_weekly_df['P'], 0)
m1_weekly_df['POST_B2C_T'] = np.where(m1_weekly_df['METRIC_CD']=='TB2R010500CS', m1_weekly_df['P'], 0)
m1_weekly_df['POST_B2C_D'] = np.where(m1_weekly_df['METRIC_CD']=='DB2R010500CS', m1_weekly_df['P'], 0)
# m1_weekly_df['POST_B2B'] = np.where(m1_weekly_df['METRIC_CD']=='B2R020500CS', m1_weekly_df['P'], 0)
# m1_weekly_df['POST_B2B_T'] = np.where(m1_weekly_df['METRIC_CD']=='TB2R020500CS', m1_weekly_df['P'], 0)
# m1_weekly_df['POST_B2B_D'] = np.where(m1_weekly_df['METRIC_CD']=='DB2R020500CS', m1_weekly_df['P'], 0)
m1_weekly_df['TOL_CONN'] = np.where(m1_weekly_df['METRIC_CD']=='TB3R000601CS', m1_weekly_df['P'], 0)
# m1_weekly_df['TOL_CONN(Install)'] = np.where(m1_weekly_df['METRIC_CD']=='TB3R000601D1CS', m1_weekly_df['P'], 0)
# m1_weekly_df['TOL_DSV'] = np.where(m1_weekly_df['METRIC_CD']=='TB3R000602CS', m1_weekly_df['P'], 0)
# m1_weekly_df['TOL_DSV(Install)'] = np.where(m1_weekly_df['METRIC_CD']=='TB3R000602D1CS', m1_weekly_df['P'], 0)
m1_weekly_df['TVS_NOW'] = np.where(m1_weekly_df['METRIC_CD']=='TB4R001700CS', m1_weekly_df['P'], 0)
# m1_weekly_df['TVS_CMDU'] = np.where(m1_weekly_df['METRIC_CD']=='TB4R001004CS', m1_weekly_df['P'], 0)

# m1_weekly_df = m1_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_weekly_df['WoW'] = m1_weekly_df['TOTAL'] - m1_weekly_df['TOTAL'].shift()
# m1_weekly_df = m1_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
# m1_weekly_df = m1_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_weekly_df = m1_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_weekly_df['WoW'] = m1_weekly_df['TOTAL'] - m1_weekly_df['TOTAL'].shift()
m1_weekly_df = m1_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
m1_weekly_df = m1_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'WoW', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_weekly_df_display = m1_weekly_df.copy()
m1_weekly_df_display['ACTUAL_AS_OF'] = m1_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_weekly_df_display[col] = m1_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,WoW,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,2025018,2025-06-17 00:53:59,20250504,0,"71,351,613","38,367,911","32,983,702","51,937,498","24,423,793","27,513,705","12,596,056","10,018,196","2,577,860","3,112,450","119,334"
1,2025019,2025-06-17 00:53:59,20250511,"47,205,504","118,557,117","67,372,373","51,184,744","87,456,449","43,643,248","43,813,201","19,633,666","14,975,619","4,658,047","7,006,724","110,012"
2,2025020,2025-06-17 00:53:59,20250518,"697,977","119,255,094","68,027,408","51,227,687","89,823,692","45,955,559","43,868,133","18,191,170","13,807,640","4,383,529","6,535,539","81,335"
3,2025021,2025-06-17 00:53:59,20250525,"-6,066,481","113,188,613","58,752,690","54,435,923","88,667,894","39,415,512","49,252,382","15,449,068","10,996,556","4,452,512","6,365,817","118,618"
4,2025022,2025-06-17 00:53:59,20250601,"13,066,467","126,255,081","63,096,573","63,158,508","94,336,188","42,119,869","52,216,319","16,149,194","12,254,532","3,894,662","6,126,899","144,967"
5,2025023,2025-06-17 00:53:59,20250608,"-17,217,291","109,037,789","58,259,037","50,778,752","80,630,278","35,660,293","44,969,985","18,753,925","14,448,640","4,305,285","6,641,204","134,362"
6,2025024,2025-06-17 00:53:59,20250615,"-16,244,924","92,792,866","52,048,073","40,744,792","60,908,290","31,318,488","35,471,283","14,764,071","13,231,046","3,639,064","6,209,757","79,314"
7,2025025,2025-06-17 00:53:59,20250615,"-92,792,866",0,0,0,0,0,0,0,0,0,0,0


In [8]:
''' Inflow M1(All Channel) Daily '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'B0R00010001CS' #Total Inflow M1
    , 'TB0R00010001CS' #Total Inflow M1 : TRUE
    , 'DB0R00010001CS' #Total Inflow M1 : DTAC

    , 'B1R000900CS' #Prepaid Inflow M1
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC

    , 'B2R010500CS' #Postpaid Inflow M1 B2C
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    # , 'B2R020500CS' #Postpaid Inflow M1 B2B
    # , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    # , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC

    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    # , 'TB3R000601D1CS' #TOL Inflow M1 Connected : Consumer (Install Location)
    # , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    # , 'TB3R000602D1CS' #TOL Inflow M1 Connected : DataService (Install Location)

    , 'TB4R001700CS' #TVS Now Inflow M1
    # , 'TB4R001004CS' #TVS CMDU Inflow M1
    ]

m1_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
m1_daily_df = m1_daily_df.loc[m1_daily_df['METRIC_CD'].isin(v_metric_list)]

m1_daily_df['TOTAL'] = np.where(m1_daily_df['METRIC_CD']=='B0R00010001CS', m1_daily_df['P'], 0)
m1_daily_df['TOTAL_T'] = np.where(m1_daily_df['METRIC_CD']=='TB0R00010001CS', m1_daily_df['P'], 0)
m1_daily_df['TOTAL_D'] = np.where(m1_daily_df['METRIC_CD']=='DB0R00010001CS', m1_daily_df['P'], 0)
m1_daily_df['PRE'] = np.where(m1_daily_df['METRIC_CD']=='B1R000900CS', m1_daily_df['P'], 0)
m1_daily_df['PRE_T'] = np.where(m1_daily_df['METRIC_CD']=='TB1R000900CS', m1_daily_df['P'], 0)
m1_daily_df['PRE_D'] = np.where(m1_daily_df['METRIC_CD']=='DB1R000900CS', m1_daily_df['P'], 0)
m1_daily_df['POST_B2C'] = np.where(m1_daily_df['METRIC_CD']=='B2R010500CS', m1_daily_df['P'], 0)
m1_daily_df['POST_B2C_T'] = np.where(m1_daily_df['METRIC_CD']=='TB2R010500CS', m1_daily_df['P'], 0)
m1_daily_df['POST_B2C_D'] = np.where(m1_daily_df['METRIC_CD']=='DB2R010500CS', m1_daily_df['P'], 0)
# m1_daily_df['POST_B2B'] = np.where(m1_daily_df['METRIC_CD']=='B2R020500CS', m1_daily_df['P'], 0)
# m1_daily_df['POST_B2B_T'] = np.where(m1_daily_df['METRIC_CD']=='TB2R020500CS', m1_daily_df['P'], 0)
# m1_daily_df['POST_B2B_D'] = np.where(m1_daily_df['METRIC_CD']=='DB2R020500CS', m1_daily_df['P'], 0)
m1_daily_df['TOL_CONN'] = np.where(m1_daily_df['METRIC_CD']=='TB3R000601CS', m1_daily_df['P'], 0)
# m1_daily_df['TOL_CONN(Install)'] = np.where(m1_daily_df['METRIC_CD']=='TB3R000601D1CS', m1_daily_df['P'], 0)
# m1_daily_df['TOL_DSV'] = np.where(m1_daily_df['METRIC_CD']=='TB3R000602CS', m1_daily_df['P'], 0)
# m1_daily_df['TOL_DSV(Install)'] = np.where(m1_daily_df['METRIC_CD']=='TB3R000602D1CS', m1_daily_df['P'], 0)
m1_daily_df['TVS_NOW'] = np.where(m1_daily_df['METRIC_CD']=='TB4R001700CS', m1_daily_df['P'], 0)
# m1_daily_df['TVS_CMDU'] = np.where(m1_daily_df['METRIC_CD']=='TB4R001004CS', m1_daily_df['P'], 0)

# m1_daily_df = m1_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_daily_df['DoD'] = m1_daily_df['TOTAL'] - m1_daily_df['TOTAL'].shift()
# m1_daily_df = m1_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
# m1_daily_df = m1_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_daily_df = m1_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_daily_df['DoD'] = m1_daily_df['TOTAL'] - m1_daily_df['TOTAL'].shift()
m1_daily_df = m1_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
m1_daily_df = m1_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'DoD', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_daily_df_display = m1_daily_df.copy()
mod_col_list = m1_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_daily_df_display[col] = m1_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,DoD,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,202506,20250616,2025-06-17 00:53:59,"-2,855,598",0,0,0,0,0,0,0,0,0,0,0
1,202506,20250615,2025-06-17 00:53:59,"-6,577,764","2,855,598","2,855,598",0,0,0,0,0,"2,106,039",0,"725,780",0
2,202506,20250614,2025-06-17 00:53:59,"-5,363,844","9,433,362","3,074,145","6,359,217",0,0,"5,881,482","2,541,541","2,074,216","467,325","955,640","10,236"
3,202506,20250613,2025-06-17 00:53:59,"-1,181,172","14,797,206","8,116,657","6,680,548","11,234,933","5,398,194","5,836,738","2,358,415","1,716,574","641,841","812,054","10,355"
4,202506,20250612,2025-06-17 00:53:59,"-2,168,846","15,978,377","8,217,456","7,760,921","11,288,623","5,370,028","5,918,595","2,324,196","1,721,942","602,254","884,633","15,718"
5,202506,20250611,2025-06-17 00:53:59,"2,054,222","18,147,224","11,739,311","6,407,913","14,548,433","8,840,505","5,707,928","2,437,204","1,794,307","642,897","886,119","7,540"
6,202506,20250610,2025-06-17 00:53:59,"604,905","16,093,002","9,333,207","6,759,795","12,197,523","6,172,727","6,024,796","2,670,125","2,002,300","667,825","972,314","13,257"
7,202506,20250609,2025-06-17 00:53:59,"310,820","15,488,097","8,711,698","6,776,398","11,638,778","5,537,034","6,101,744","2,432,591","1,815,669","616,922","973,217","22,208"
8,202506,20250608,2025-06-17 00:53:59,"52,672","15,177,277","8,427,386","6,749,891","11,467,060","5,195,049","6,272,012","2,722,726","2,267,985","454,741","929,362","18,677"
9,202506,20250607,2025-06-17 00:53:59,"-957,396","15,124,605","8,179,976","6,944,628","11,540,853","5,122,763","6,418,090","2,559,244","2,058,334","500,910","937,717","12,836"


## Overview(Geo) by Period

In [9]:
''' Inflow M1(Geo Channel) Yearly '''

v_metric_list = [
    'B0R00010001CG' #Total Inflow M1 - GEO Channel
    , 'TB0R00010001CG' #Total Inflow M1 : TRUE - GEO Channel
    , 'DB0R00010001CG' #Total Inflow M1 : DTAC - GEO Channel

    , 'B1R000900CG' #Prepaid Inflow M1 - GEO Channel
    , 'TB1R000900CG' #Prepaid Inflow M1 : TMH - GEO Channel
    , 'DB1R000900CG' #Prepaid Inflow M1 : DTAC - GEO Channel
    
    , 'B2R010500CG' #Postpaid Inflow M1 B2C - GEO Channel
    , 'TB2R010500CG' #Postpaid Inflow M1 B2C : TMH - GEO Channel
    , 'DB2R010500CG' #Postpaid Inflow M1 B2C : DTAC - GEO Channel
    # , 'B2R020500CG' #Postpaid Inflow M1 B2B - GEO Channel
    # , 'TB2R020500CG' #Postpaid Inflow M1 B2B : TMH - GEO Channel
    # , 'DB2R020500CG' #Postpaid Inflow M1 B2B : DTAC - GEO Channel
    
    , 'TB3R000601CG' #TOL Inflow M1 Connected : Consumer - GEO Channel
    # , 'TB3R000601D1CG' #TOL Inflow M1 Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3R000602CG' #TOL Inflow M1 Connected : DataService - GEO Channel
    # , 'TB3R000602D1CG' #TOL Inflow M1 Connected : DataService - GEO Channel (Install Location)

    , 'TB4R001700CG' #TVS Now Inflow M1 - GEO Channel
    # , 'TB4R001004CG' #TVS CMDU Inflow M1 - GEO Channel
    ]

m1_geo_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
m1_geo_yearly_df = m1_geo_yearly_df.loc[m1_geo_yearly_df['METRIC_CD'].isin(v_metric_list)]

m1_geo_yearly_df['TOTAL'] = np.where(m1_geo_yearly_df['METRIC_CD']=='B0R00010001CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['TOTAL_T'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB0R00010001CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['TOTAL_D'] = np.where(m1_geo_yearly_df['METRIC_CD']=='DB0R00010001CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['PRE'] = np.where(m1_geo_yearly_df['METRIC_CD']=='B1R000900CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['PRE_T'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB1R000900CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['PRE_D'] = np.where(m1_geo_yearly_df['METRIC_CD']=='DB1R000900CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['POST_B2C'] = np.where(m1_geo_yearly_df['METRIC_CD']=='B2R010500CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['POST_B2C_T'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB2R010500CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['POST_B2C_D'] = np.where(m1_geo_yearly_df['METRIC_CD']=='DB2R010500CG', m1_geo_yearly_df['P'], 0)
# m1_geo_yearly_df['POST_B2B'] = np.where(m1_geo_yearly_df['METRIC_CD']=='B2R020500CG', m1_geo_yearly_df['P'], 0)
# m1_geo_yearly_df['POST_B2B_T'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB2R020500CG', m1_geo_yearly_df['P'], 0)
# m1_geo_yearly_df['POST_B2B_D'] = np.where(m1_geo_yearly_df['METRIC_CD']=='DB2R020500CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['TOL_CONN'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB3R000601CG', m1_geo_yearly_df['P'], 0)
# m1_geo_yearly_df['TOL_CONN(Install)'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB3R000601D1CG', m1_geo_yearly_df['P'], 0)
# m1_geo_yearly_df['TOL_DSV'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB3R000602CG', m1_geo_yearly_df['P'], 0)
# m1_geo_yearly_df['TOL_DSV(Install)'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB3R000602D1CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['TVS_NOW'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB4R001700CG', m1_geo_yearly_df['P'], 0)
# m1_geo_yearly_df['TVS_CMDU'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB4R001004CG', m1_geo_yearly_df['P'], 0)

# m1_geo_yearly_df = m1_geo_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_geo_yearly_df['YoY'] = m1_geo_yearly_df['TOTAL'] - m1_geo_yearly_df['TOTAL'].shift()
# m1_geo_yearly_df = m1_geo_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
# m1_geo_yearly_df = m1_geo_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'YoY', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_geo_yearly_df = m1_geo_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_geo_yearly_df['YoY'] = m1_geo_yearly_df['TOTAL'] - m1_geo_yearly_df['TOTAL'].shift()
m1_geo_yearly_df = m1_geo_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
m1_geo_yearly_df = m1_geo_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'YoY', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_geo_yearly_df_display = m1_geo_yearly_df.copy()
m1_geo_yearly_df_display['ACTUAL_AS_OF'] = m1_geo_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_geo_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_geo_yearly_df_display[col] = m1_geo_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_geo_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,YoY,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,2025,2025-06-17 00:53:59,20250615,0,"2,655,327,104","1,383,134,760","1,272,192,343","2,286,919,698","1,063,317,254","1,229,369,673","223,445,195","183,791,362","40,628,942","116,504,999","2,339,712"


In [10]:
''' Inflow M1(Geo Channel) Quarterly '''

v_metric_list = [
    'B0R00010001CG' #Total Inflow M1 - GEO Channel
    , 'TB0R00010001CG' #Total Inflow M1 : TRUE - GEO Channel
    , 'DB0R00010001CG' #Total Inflow M1 : DTAC - GEO Channel

    , 'B1R000900CG' #Prepaid Inflow M1 - GEO Channel
    , 'TB1R000900CG' #Prepaid Inflow M1 : TMH - GEO Channel
    , 'DB1R000900CG' #Prepaid Inflow M1 : DTAC - GEO Channel
    
    , 'B2R010500CG' #Postpaid Inflow M1 B2C - GEO Channel
    , 'TB2R010500CG' #Postpaid Inflow M1 B2C : TMH - GEO Channel
    , 'DB2R010500CG' #Postpaid Inflow M1 B2C : DTAC - GEO Channel
    # , 'B2R020500CG' #Postpaid Inflow M1 B2B - GEO Channel
    # , 'TB2R020500CG' #Postpaid Inflow M1 B2B : TMH - GEO Channel
    # , 'DB2R020500CG' #Postpaid Inflow M1 B2B : DTAC - GEO Channel
    
    , 'TB3R000601CG' #TOL Inflow M1 Connected : Consumer - GEO Channel
    # , 'TB3R000601D1CG' #TOL Inflow M1 Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3R000602CG' #TOL Inflow M1 Connected : DataService - GEO Channel
    # , 'TB3R000602D1CG' #TOL Inflow M1 Connected : DataService - GEO Channel (Install Location)

    , 'TB4R001700CG' #TVS Now Inflow M1 - GEO Channel
    # , 'TB4R001004CG' #TVS CMDU Inflow M1 - GEO Channel
    ]

m1_geo_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
m1_geo_quarterly_df = m1_geo_quarterly_df.loc[m1_geo_quarterly_df['METRIC_CD'].isin(v_metric_list)]

m1_geo_quarterly_df['TOTAL'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='B0R00010001CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['TOTAL_T'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB0R00010001CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['TOTAL_D'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='DB0R00010001CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['PRE'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='B1R000900CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['PRE_T'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB1R000900CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['PRE_D'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='DB1R000900CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['POST_B2C'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='B2R010500CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['POST_B2C_T'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB2R010500CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['POST_B2C_D'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='DB2R010500CG', m1_geo_quarterly_df['P'], 0)
# m1_geo_quarterly_df['POST_B2B'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='B2R020500CG', m1_geo_quarterly_df['P'], 0)
# m1_geo_quarterly_df['POST_B2B_T'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB2R020500CG', m1_geo_quarterly_df['P'], 0)
# m1_geo_quarterly_df['POST_B2B_D'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='DB2R020500CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['TOL_CONN'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB3R000601CG', m1_geo_quarterly_df['P'], 0)
# m1_geo_quarterly_df['TOL_CONN(Install)'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB3R000601D1CG', m1_geo_quarterly_df['P'], 0)
# m1_geo_quarterly_df['TOL_DSV'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB3R000602CG', m1_geo_quarterly_df['P'], 0)
# m1_geo_quarterly_df['TOL_DSV(Install)'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB3R000602D1CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['TVS_NOW'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB4R001700CG', m1_geo_quarterly_df['P'], 0)
# m1_geo_quarterly_df['TVS_CMDU'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB4R001004CG', m1_geo_quarterly_df['P'], 0)

# m1_geo_quarterly_df = m1_geo_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_geo_quarterly_df['QoQ'] = m1_geo_quarterly_df['TOTAL'] - m1_geo_quarterly_df['TOTAL'].shift()
# m1_geo_quarterly_df = m1_geo_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
# m1_geo_quarterly_df = m1_geo_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'QoQ', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_geo_quarterly_df = m1_geo_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_geo_quarterly_df['QoQ'] = m1_geo_quarterly_df['TOTAL'] - m1_geo_quarterly_df['TOTAL'].shift()
m1_geo_quarterly_df = m1_geo_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
m1_geo_quarterly_df = m1_geo_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'QoQ', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_geo_quarterly_df_display = m1_geo_quarterly_df.copy()
m1_geo_quarterly_df_display['ACTUAL_AS_OF'] = m1_geo_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_geo_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_geo_quarterly_df_display[col] = m1_geo_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_geo_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,QoQ,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,20251,2025-06-17 00:53:59,20250331,0,"1,550,772,572","790,266,697","760,505,875","1,351,430,890","614,011,326","737,419,564","124,890,149","102,959,079","21,931,070","62,397,203","1,111,862"
1,20252,2025-06-17 00:53:59,20250615,"-446,218,039","1,104,554,532","592,868,063","511,686,469","935,488,808","449,305,928","491,950,109","98,555,046","80,832,283","18,697,872","54,107,796","1,227,851"


In [11]:
''' Inflow M1(Geo Channel) Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
    'B0R00010001CG' #Total Inflow M1 - GEO Channel
    , 'TB0R00010001CG' #Total Inflow M1 : TRUE - GEO Channel
    , 'DB0R00010001CG' #Total Inflow M1 : DTAC - GEO Channel

    , 'B1R000900CG' #Prepaid Inflow M1 - GEO Channel
    , 'TB1R000900CG' #Prepaid Inflow M1 : TMH - GEO Channel
    , 'DB1R000900CG' #Prepaid Inflow M1 : DTAC - GEO Channel
    
    , 'B2R010500CG' #Postpaid Inflow M1 B2C - GEO Channel
    , 'TB2R010500CG' #Postpaid Inflow M1 B2C : TMH - GEO Channel
    , 'DB2R010500CG' #Postpaid Inflow M1 B2C : DTAC - GEO Channel
    # , 'B2R020500CG' #Postpaid Inflow M1 B2B - GEO Channel
    # , 'TB2R020500CG' #Postpaid Inflow M1 B2B : TMH - GEO Channel
    # , 'DB2R020500CG' #Postpaid Inflow M1 B2B : DTAC - GEO Channel
    
    , 'TB3R000601CG' #TOL Inflow M1 Connected : Consumer - GEO Channel
    # , 'TB3R000601D1CG' #TOL Inflow M1 Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3R000602CG' #TOL Inflow M1 Connected : DataService - GEO Channel
    # , 'TB3R000602D1CG' #TOL Inflow M1 Connected : DataService - GEO Channel (Install Location)

    , 'TB4R001700CG' #TVS Now Inflow M1 - GEO Channel
    # , 'TB4R001004CG' #TVS CMDU Inflow M1 - GEO Channel
    ]

m1_geo_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr].copy()
m1_geo_monthly_df = m1_geo_monthly_df.loc[m1_geo_monthly_df['METRIC_CD'].isin(v_metric_list)]

m1_geo_monthly_df['TOTAL'] = np.where(m1_geo_monthly_df['METRIC_CD']=='B0R00010001CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['TOTAL_T'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB0R00010001CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['TOTAL_D'] = np.where(m1_geo_monthly_df['METRIC_CD']=='DB0R00010001CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['PRE'] = np.where(m1_geo_monthly_df['METRIC_CD']=='B1R000900CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['PRE_T'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB1R000900CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['PRE_D'] = np.where(m1_geo_monthly_df['METRIC_CD']=='DB1R000900CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['POST_B2C'] = np.where(m1_geo_monthly_df['METRIC_CD']=='B2R010500CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['POST_B2C_T'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB2R010500CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['POST_B2C_D'] = np.where(m1_geo_monthly_df['METRIC_CD']=='DB2R010500CG', m1_geo_monthly_df['P'], 0)
# m1_geo_monthly_df['POST_B2B'] = np.where(m1_geo_monthly_df['METRIC_CD']=='B2R020500CG', m1_geo_monthly_df['P'], 0)
# m1_geo_monthly_df['POST_B2B_T'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB2R020500CG', m1_geo_monthly_df['P'], 0)
# m1_geo_monthly_df['POST_B2B_D'] = np.where(m1_geo_monthly_df['METRIC_CD']=='DB2R020500CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['TOL_CONN'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB3R000601CG', m1_geo_monthly_df['P'], 0)
# m1_geo_monthly_df['TOL_CONN(Install)'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB3R000601D1CG', m1_geo_monthly_df['P'], 0)
# m1_geo_monthly_df['TOL_DSV'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB3R000602CG', m1_geo_monthly_df['P'], 0)
# m1_geo_monthly_df['TOL_DSV(Install)'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB3R000602D1CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['TVS_NOW'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB4R001700CG', m1_geo_monthly_df['P'], 0)
# m1_geo_monthly_df['TVS_CMDU'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB4R001004CG', m1_geo_monthly_df['P'], 0)

# m1_geo_monthly_df = m1_geo_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_geo_monthly_df['MoM'] = m1_geo_monthly_df['TOTAL'] - m1_geo_monthly_df['TOTAL'].shift()
# m1_geo_monthly_df = m1_geo_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
# m1_geo_monthly_df = m1_geo_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'MoM', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_geo_monthly_df = m1_geo_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_geo_monthly_df['MoM'] = m1_geo_monthly_df['TOTAL'] - m1_geo_monthly_df['TOTAL'].shift()
m1_geo_monthly_df = m1_geo_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
m1_geo_monthly_df = m1_geo_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'MoM', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_geo_monthly_df_display = m1_geo_monthly_df.copy()
m1_geo_monthly_df_display['ACTUAL_AS_OF'] = m1_geo_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_geo_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_geo_monthly_df_display[col] = m1_geo_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_geo_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,MoM,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,202501,2025-06-17 00:53:59,20250131,0,"560,918,001","278,217,598","282,700,403","498,995,206","223,753,635","275,241,571","38,555,159","31,477,331","7,077,828","19,471,134","376,664"
1,202502,2025-06-17 00:53:59,20250228,"-66,452,337","494,465,664","247,017,359","247,448,304","430,617,352","190,565,264","240,052,088","39,986,117","32,971,912","7,014,205","19,961,102","264,749"
2,202503,2025-06-17 00:53:59,20250331,"923,243","495,388,907","265,031,740","230,357,167","421,818,332","199,692,427","222,125,906","46,348,874","38,509,836","7,839,038","22,964,967","470,449"
3,202504,2025-06-17 00:53:59,20250430,"-36,710,304","458,678,603","251,562,458","207,116,145","388,672,340","189,643,908","199,028,432","45,183,640","37,483,652","7,699,989","20,901,734","441,246"
4,202505,2025-06-17 00:53:59,20250531,"732,109","459,410,712","244,316,516","215,094,197","395,787,628","188,609,826","207,177,802","36,970,047","29,453,492","7,516,555","22,364,854","556,079"
5,202506,2025-06-17 00:53:59,20250615,"-272,945,496","186,465,216","96,989,090","89,476,126","151,028,840","71,052,194","85,743,875","16,401,358","13,895,139","3,481,328","10,841,208","230,525"


In [12]:
''' Inflow M1(Geo Channel) Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'B0R00010001CG' #Total Inflow M1 - GEO Channel
    , 'TB0R00010001CG' #Total Inflow M1 : TRUE - GEO Channel
    , 'DB0R00010001CG' #Total Inflow M1 : DTAC - GEO Channel

    , 'B1R000900CG' #Prepaid Inflow M1 - GEO Channel
    , 'TB1R000900CG' #Prepaid Inflow M1 : TMH - GEO Channel
    , 'DB1R000900CG' #Prepaid Inflow M1 : DTAC - GEO Channel
    
    , 'B2R010500CG' #Postpaid Inflow M1 B2C - GEO Channel
    , 'TB2R010500CG' #Postpaid Inflow M1 B2C : TMH - GEO Channel
    , 'DB2R010500CG' #Postpaid Inflow M1 B2C : DTAC - GEO Channel
    # , 'B2R020500CG' #Postpaid Inflow M1 B2B - GEO Channel
    # , 'TB2R020500CG' #Postpaid Inflow M1 B2B : TMH - GEO Channel
    # , 'DB2R020500CG' #Postpaid Inflow M1 B2B : DTAC - GEO Channel
    
    , 'TB3R000601CG' #TOL Inflow M1 Connected : Consumer - GEO Channel
    # , 'TB3R000601D1CG' #TOL Inflow M1 Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3R000602CG' #TOL Inflow M1 Connected : DataService - GEO Channel
    # , 'TB3R000602D1CG' #TOL Inflow M1 Connected : DataService - GEO Channel (Install Location)

    , 'TB4R001700CG' #TVS Now Inflow M1 - GEO Channel
    # , 'TB4R001004CG' #TVS CMDU Inflow M1 - GEO Channel
    ]

m1_geo_weekly_df = chk_src_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
m1_geo_weekly_df = m1_geo_weekly_df.loc[m1_geo_weekly_df['METRIC_CD'].isin(v_metric_list)]

m1_geo_weekly_df['TOTAL'] = np.where(m1_geo_weekly_df['METRIC_CD']=='B0R00010001CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['TOTAL_T'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB0R00010001CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['TOTAL_D'] = np.where(m1_geo_weekly_df['METRIC_CD']=='DB0R00010001CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['PRE'] = np.where(m1_geo_weekly_df['METRIC_CD']=='B1R000900CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['PRE_T'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB1R000900CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['PRE_D'] = np.where(m1_geo_weekly_df['METRIC_CD']=='DB1R000900CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['POST_B2C'] = np.where(m1_geo_weekly_df['METRIC_CD']=='B2R010500CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['POST_B2C_T'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB2R010500CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['POST_B2C_D'] = np.where(m1_geo_weekly_df['METRIC_CD']=='DB2R010500CG', m1_geo_weekly_df['P'], 0)
# m1_geo_weekly_df['POST_B2B'] = np.where(m1_geo_weekly_df['METRIC_CD']=='B2R020500CG', m1_geo_weekly_df['P'], 0)
# m1_geo_weekly_df['POST_B2B_T'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB2R020500CG', m1_geo_weekly_df['P'], 0)
# m1_geo_weekly_df['POST_B2B_D'] = np.where(m1_geo_weekly_df['METRIC_CD']=='DB2R020500CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['TOL_CONN'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB3R000601CG', m1_geo_weekly_df['P'], 0)
# m1_geo_weekly_df['TOL_CONN(Install)'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB3R000601D1CG', m1_geo_weekly_df['P'], 0)
# m1_geo_weekly_df['TOL_DSV'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB3R000602CG', m1_geo_weekly_df['P'], 0)
# m1_geo_weekly_df['TOL_DSV(Install)'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB3R000602D1CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['TVS_NOW'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB4R001700CG', m1_geo_weekly_df['P'], 0)
# m1_geo_weekly_df['TVS_CMDU'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB4R001004CG', m1_geo_weekly_df['P'], 0)

# m1_geo_weekly_df = m1_geo_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_geo_weekly_df['WoW'] = m1_geo_weekly_df['TOTAL'] - m1_geo_weekly_df['TOTAL'].shift()
# m1_geo_weekly_df = m1_geo_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
# m1_geo_weekly_df = m1_geo_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'WoW', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_geo_weekly_df = m1_geo_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_geo_weekly_df['WoW'] = m1_geo_weekly_df['TOTAL'] - m1_geo_weekly_df['TOTAL'].shift()
m1_geo_weekly_df = m1_geo_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
m1_geo_weekly_df = m1_geo_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'WoW', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_geo_weekly_df_display = m1_geo_weekly_df.copy()
m1_geo_weekly_df_display['ACTUAL_AS_OF'] = m1_geo_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_geo_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_geo_weekly_df_display[col] = m1_geo_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_geo_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,WoW,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,2025018,2025-06-17 00:53:59,20250504,0,"60,402,874","32,043,527","28,359,347","51,218,481","24,152,685","27,065,796","6,334,713","5,077,813","1,256,900","2,506,557","119,334"
1,2025019,2025-06-17 00:53:59,20250511,"41,774,422","102,177,296","57,045,097","45,132,200","86,433,048","43,125,564","43,307,484","9,377,275","7,638,790","1,738,485","5,640,799","109,900"
2,2025020,2025-06-17 00:53:59,20250518,"1,254,324","103,431,620","58,308,394","45,123,226","88,873,567","45,506,441","43,367,126","8,481,078","6,811,667","1,669,411","5,150,972","81,335"
3,2025021,2025-06-17 00:53:59,20250525,"-2,703,452","100,728,168","50,436,910","50,291,259","87,680,207","39,068,345","48,611,863","6,977,220","5,425,771","1,551,449","4,970,769","118,087"
4,2025022,2025-06-17 00:53:59,20250601,"5,959,002","106,687,170","53,225,355","53,461,815","93,429,983","41,749,391","51,680,592","7,184,152","5,496,646","1,687,506","4,774,070","144,716"
5,2025023,2025-06-17 00:53:59,20250608,"-12,937,487","93,749,684","47,661,332","46,088,352","79,433,592","35,316,994","44,116,598","8,320,369","6,493,944","1,826,425","5,243,880","134,028"
6,2025024,2025-06-17 00:53:59,20250615,"-15,050,567","78,699,116","42,584,991","36,114,125","59,747,591","30,742,601","34,772,219","6,696,599","6,404,001","1,267,707","4,919,015","79,203"
7,2025025,2025-06-17 00:53:59,20250615,"-78,699,116",0,0,0,0,0,0,0,0,0,0,0


In [13]:
''' Inflow M1(Geo Channel) Daily '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'B0R00010001CG' #Total Inflow M1 - GEO Channel
    , 'TB0R00010001CG' #Total Inflow M1 : TRUE - GEO Channel
    , 'DB0R00010001CG' #Total Inflow M1 : DTAC - GEO Channel

    , 'B1R000900CG' #Prepaid Inflow M1 - GEO Channel
    , 'TB1R000900CG' #Prepaid Inflow M1 : TMH - GEO Channel
    , 'DB1R000900CG' #Prepaid Inflow M1 : DTAC - GEO Channel
    
    , 'B2R010500CG' #Postpaid Inflow M1 B2C - GEO Channel
    , 'TB2R010500CG' #Postpaid Inflow M1 B2C : TMH - GEO Channel
    , 'DB2R010500CG' #Postpaid Inflow M1 B2C : DTAC - GEO Channel
    # , 'B2R020500CG' #Postpaid Inflow M1 B2B - GEO Channel
    # , 'TB2R020500CG' #Postpaid Inflow M1 B2B : TMH - GEO Channel
    # , 'DB2R020500CG' #Postpaid Inflow M1 B2B : DTAC - GEO Channel
    
    , 'TB3R000601CG' #TOL Inflow M1 Connected : Consumer - GEO Channel
    # , 'TB3R000601D1CG' #TOL Inflow M1 Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3R000602CG' #TOL Inflow M1 Connected : DataService - GEO Channel
    # , 'TB3R000602D1CG' #TOL Inflow M1 Connected : DataService - GEO Channel (Install Location)

    , 'TB4R001700CG' #TVS Now Inflow M1 - GEO Channel
    # , 'TB4R001004CG' #TVS CMDU Inflow M1 - GEO Channel
    ]

m1_geo_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
m1_geo_daily_df = m1_geo_daily_df.loc[m1_geo_daily_df['METRIC_CD'].isin(v_metric_list)]

m1_geo_daily_df['TOTAL'] = np.where(m1_geo_daily_df['METRIC_CD']=='B0R00010001CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['TOTAL_T'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB0R00010001CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['TOTAL_D'] = np.where(m1_geo_daily_df['METRIC_CD']=='DB0R00010001CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['PRE'] = np.where(m1_geo_daily_df['METRIC_CD']=='B1R000900CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['PRE_T'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB1R000900CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['PRE_D'] = np.where(m1_geo_daily_df['METRIC_CD']=='DB1R000900CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['POST_B2C'] = np.where(m1_geo_daily_df['METRIC_CD']=='B2R010500CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['POST_B2C_T'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB2R010500CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['POST_B2C_D'] = np.where(m1_geo_daily_df['METRIC_CD']=='DB2R010500CG', m1_geo_daily_df['P'], 0)
# m1_geo_daily_df['POST_B2B'] = np.where(m1_geo_daily_df['METRIC_CD']=='B2R020500CG', m1_geo_daily_df['P'], 0)
# m1_geo_daily_df['POST_B2B_T'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB2R020500CG', m1_geo_daily_df['P'], 0)
# m1_geo_daily_df['POST_B2B_D'] = np.where(m1_geo_daily_df['METRIC_CD']=='DB2R020500CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['TOL_CONN'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB3R000601CG', m1_geo_daily_df['P'], 0)
# m1_geo_daily_df['TOL_CONN(Install)'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB3R000601D1CG', m1_geo_daily_df['P'], 0)
# m1_geo_daily_df['TOL_DSV'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB3R000602CG', m1_geo_daily_df['P'], 0)
# m1_geo_daily_df['TOL_DSV(Install)'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB3R000602D1CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['TVS_NOW'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB4R001700CG', m1_geo_daily_df['P'], 0)
# m1_geo_daily_df['TVS_CMDU'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB4R001004CG', m1_geo_daily_df['P'], 0)

# m1_geo_daily_df = m1_geo_daily_df.groupby('TM_KEY_MTH', 'TM_KEY_DAY').agg({'PPN_TM':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_geo_daily_df['DoD'] = m1_geo_daily_df['TOTAL'] - m1_geo_daily_df['TOTAL'].shift()
# m1_geo_daily_df = m1_geo_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
# m1_geo_daily_df = m1_geo_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'DoD', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_geo_daily_df = m1_geo_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_geo_daily_df['DoD'] = m1_geo_daily_df['TOTAL'] - m1_geo_daily_df['TOTAL'].shift()
m1_geo_daily_df = m1_geo_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
m1_geo_daily_df = m1_geo_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'DoD', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_geo_daily_df_display = m1_geo_daily_df.copy()
mod_col_list = m1_geo_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_geo_daily_df_display[col] = m1_geo_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_geo_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,DoD,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,202506,20250616,2025-06-17 00:53:59,"-1,515,784",0,0,0,0,0,0,0,0,0,0,0
1,202506,20250615,2025-06-17 00:53:59,"-6,193,300","1,515,784","1,515,784",0,0,0,0,0,"975,108",0,"526,492",0
2,202506,20250614,2025-06-17 00:53:59,"-5,225,967","7,709,085","1,714,213","5,994,872",0,0,"5,767,229","1,185,348","967,140","218,208","716,575","10,236"
3,202506,20250613,2025-06-17 00:53:59,"-36,305","12,935,052","6,967,306","5,967,746","11,069,423","5,326,477","5,742,946","1,076,662","865,615","211,047","669,827","10,244"
4,202506,20250612,2025-06-17 00:53:59,"-3,155,806","12,971,356","6,948,534","6,022,822","11,113,710","5,307,458","5,806,252","1,002,905","808,589","194,316","731,039","15,718"
5,202506,20250611,2025-06-17 00:53:59,"2,000,745","16,127,162","10,336,083","5,791,079","14,267,194","8,677,045","5,590,149","1,073,306","882,159","191,147","707,116","7,540"
6,202506,20250610,2025-06-17 00:53:59,"812,156","14,126,417","7,990,121","6,136,295","11,943,968","6,056,439","5,887,529","1,290,805","1,050,856","239,949","788,719","13,257"
7,202506,20250609,2025-06-17 00:53:59,"76,051","13,314,261","7,112,950","6,201,310","11,353,296","5,375,182","5,978,113","1,067,574","854,534","213,040","779,247","22,208"
8,202506,20250608,2025-06-17 00:53:59,"-37,426","13,238,210","6,744,980","6,493,230","11,250,238","5,088,917","6,161,321","1,252,096","943,325","308,771","685,093","18,677"
9,202506,20250607,2025-06-17 00:53:59,"-808,082","13,275,636","6,738,978","6,536,658","11,372,331","5,069,056","6,303,275","1,134,297","925,043","209,254","708,999","12,725"


## Products Summary

### Prep Monthly Data

In [14]:
''' Monthly Summary '''

monthly_df = chk_src_df.copy()
monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
# monthly_df['DIFF (C-P)'] = monthly_df['C'] - monthly_df['P']

monthly_df_display = monthly_df.copy()
mod_col_list = monthly_df_display.iloc[:, 5:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

# monthly_df_display
# monthly_df_display.loc[monthly_df_display['TM_KEY_MTH']==202501]

In [15]:
''' Parameter '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth
# v_tm_key_mth = 202505

print(f'v_tm_key_mth: {v_tm_key_mth}')

v_tm_key_mth: 202505


### All Service

In [16]:
''' All Service (All Channel) '''

v_product_grp = 'All Services'

all_service_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp].copy()
all_service_df = all_service_df.loc[~all_service_df['METRIC_NAME'].str.contains('GEO')]
all_service_df = all_service_df.loc[all_service_df['TM_KEY_MTH']==v_tm_key_mth]
all_service_df = all_service_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
all_service_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202505,All Services,B0R00010001CS,Total Inflow M1,2025-06-17 00:53:59,"532,626,295","532,626,295","441,017,881","441,108,097","434,991,221"
1,202505,All Services,DB0R00010001CS,Total Inflow M1 : DTAC,2025-06-17 00:53:59,"245,392,131","245,392,131","227,535,299","227,535,299","224,811,447"
2,202505,All Services,TB0R00010001CS,Total Inflow M1 : TRUE,2025-06-17 00:53:59,"287,234,164","287,234,164","213,482,582","213,572,798","210,179,774"


In [17]:
''' All Service (Geo Channel) '''

v_product_grp = 'All Services'

all_service_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp].copy()
all_service_geo_df = all_service_geo_df.loc[all_service_geo_df['METRIC_NAME'].str.contains('GEO')]
all_service_geo_df = all_service_geo_df.loc[all_service_geo_df['TM_KEY_MTH']==v_tm_key_mth]
all_service_geo_df = all_service_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
all_service_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202505,All Services,B0R00010001CG,Total Inflow M1 - GEO Channel,2025-06-17 00:53:59,0,"459,410,712","407,881,327","407,971,543","401,858,889"
1,202505,All Services,DB0R00010001CG,Total Inflow M1 : DTAC - GEO Channel,2025-06-17 00:53:59,0,"215,094,197","211,803,486","211,803,486","209,083,856"
2,202505,All Services,TB0R00010001CG,Total Inflow M1 : TRUE - GEO Channel,2025-06-17 00:53:59,0,"244,316,516","196,077,841","196,168,057","192,775,033"


### Prepaid

In [18]:
''' Prepaid (All Channel) '''

v_product_grp = 'Prepaid'

prepaid_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
prepaid_df = prepaid_df.loc[~prepaid_df['METRIC_NAME'].str.contains('GEO')]
prepaid_df = prepaid_df.loc[prepaid_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_df = prepaid_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
prepaid_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202505,Prepaid,B1R000900CS,Prepaid Inflow M1,2025-06-17 00:53:59,"400,164,161","400,164,161","346,583,592","346,583,592","340,682,383"
1,202505,Prepaid,DB1R000900CS,Prepaid Inflow M1 : DTAC,2025-06-17 00:53:59,"209,649,746","209,649,746","206,212,362","206,212,362","203,488,111"
2,202505,Prepaid,TB1R000900CS,Prepaid Inflow M1 : TMH,2025-06-17 00:53:59,"190,514,414","190,514,414","140,371,230","140,371,230","137,194,272"


In [19]:
''' Prepaid (Geo Channel) '''

v_product_grp = 'Prepaid'

prepaid_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
prepaid_geo_df = prepaid_geo_df.loc[prepaid_geo_df['METRIC_NAME'].str.contains('GEO')]
prepaid_geo_df = prepaid_geo_df.loc[prepaid_geo_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_geo_df = prepaid_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
prepaid_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202505,Prepaid,B1R000900CG,Prepaid Inflow M1 - GEO Channel,2025-06-17 00:53:59,0,"395,787,628","344,268,514","344,268,514","338,371,926"
1,202505,Prepaid,DB1R000900CG,Prepaid Inflow M1 : DTAC - GEO Channel,2025-06-17 00:53:59,0,"207,177,802","203,903,918","203,903,918","201,184,288"
2,202505,Prepaid,TB1R000900CG,Prepaid Inflow M1 : TMH - GEO Channel,2025-06-17 00:53:59,0,"188,609,826","140,364,596","140,364,596","137,187,638"


### Postpaid

In [20]:
''' Postpaid B2C (All Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2c_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['METRIC_NAME'].str.contains('B2C')]
postpaid_b2c_df = postpaid_b2c_df.loc[~postpaid_b2c_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2c_df = postpaid_b2c_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202505,Postpaid,B2R010500CS,Postpaid Inflow M1 B2C,2025-06-17 00:53:59,"79,167,109","79,167,109","65,926,353","65,926,353","65,927,650"
1,202505,Postpaid,DB2R010500CS,Postpaid Inflow M1 B2C : DTAC,2025-06-17 00:53:59,"19,443,366","19,443,366","19,310,231","19,310,231","19,310,630"
2,202505,Postpaid,TB2R010500CS,Postpaid Inflow M1 B2C : TMH,2025-06-17 00:53:59,"59,723,743","59,723,743","46,616,122","46,616,122","46,617,020"


In [21]:
''' Postpaid B2C (Geo Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2c_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.loc[postpaid_b2c_geo_df['METRIC_NAME'].str.contains('B2C')]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.loc[postpaid_b2c_geo_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.loc[postpaid_b2c_geo_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2c_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202505,Postpaid,B2R010500CG,Postpaid Inflow M1 B2C - GEO Channel,2025-06-17 00:53:59,0,"36,970,047","36,951,755","36,420,150","35,958,700"
1,202505,Postpaid,DB2R010500CG,Postpaid Inflow M1 B2C : DTAC - GEO Channel,2025-06-17 00:53:59,0,"7,516,555","7,499,728","7,499,728","7,499,728"
2,202505,Postpaid,TB2R010500CG,Postpaid Inflow M1 B2C : TMH - GEO Channel,2025-06-17 00:53:59,0,"29,453,492","29,452,027","29,452,027","29,452,925"


In [22]:
''' Postpaid B2B (All Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2b_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['METRIC_NAME'].str.contains('B2B')]
postpaid_b2b_df = postpaid_b2b_df.loc[~postpaid_b2b_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2b_df = postpaid_b2b_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2b_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202505,Postpaid,B2R020500CS,Postpaid Inflow M1 B2B,2025-06-17 00:53:59,"22,538,678","22,538,678","5,138,327","5,021,050","4,638,294"
1,202505,Postpaid,DB2R020500CS,Postpaid Inflow M1 B2B : DTAC,2025-06-17 00:53:59,"16,299,019","16,299,019","2,012,706","2,012,706","2,012,706"
2,202505,Postpaid,TB2R020500CS,Postpaid Inflow M1 B2B : TMH,2025-06-17 00:53:59,"6,239,659","6,239,659","3,125,621","3,185,876","3,186,577"


In [23]:
''' Postpaid B2B (Geo Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2b_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.loc[postpaid_b2b_geo_df['METRIC_NAME'].str.contains('B2B')]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.loc[postpaid_b2b_geo_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.loc[postpaid_b2b_geo_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2b_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202505,Postpaid,B2R020500CG,Postpaid Inflow M1 B2B - GEO Channel,2025-06-17 00:53:59,0,"3,510,141","3,520,169","3,016,320","3,148,832"
1,202505,Postpaid,DB2R020500CG,Postpaid Inflow M1 B2B : DTAC - GEO Channel,2025-06-17 00:53:59,0,"399,840","399,840","399,840","399,840"
2,202505,Postpaid,TB2R020500CG,Postpaid Inflow M1 B2B : TMH - GEO Channel,2025-06-17 00:53:59,0,"3,110,301","3,120,329","3,180,584","3,181,285"


### TOL

In [24]:
''' TOL Connected : Consumer (All Channel) '''

v_product_grp = 'TOL'

tol_conn_con_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_con_df = tol_conn_con_df.loc[tol_conn_con_df['METRIC_NAME'].str.contains('Consumer')]
tol_conn_con_df = tol_conn_con_df.loc[~tol_conn_con_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_con_df = tol_conn_con_df.loc[tol_conn_con_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_con_df = tol_conn_con_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_con_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202505,TOL,TB3R000601CS,TOL Inflow M1 Connected : Consumer,2025-06-17 00:53:59,"28,260,212","28,260,212","22,365,151","22,365,151","22,141,212"
1,202505,TOL,TB3R000601D1CS,TOL Inflow M1 Connected : Consumer (Install Lo...,2025-06-17 00:53:59,"28,260,212","28,260,212","28,256,745","28,256,745","28,256,745"


In [25]:
''' TOL Connected : Consumer (Geo Channel) '''

v_product_grp = 'TOL'

tol_conn_con_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_con_geo_df = tol_conn_con_geo_df.loc[tol_conn_con_geo_df['METRIC_NAME'].str.contains('Consumer')]
tol_conn_con_geo_df = tol_conn_con_geo_df.loc[tol_conn_con_geo_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_con_geo_df = tol_conn_con_geo_df.loc[tol_conn_con_geo_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_con_geo_df = tol_conn_con_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_con_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202505,TOL,TB3R000601CG,TOL Inflow M1 Connected : Consumer - GEO Channel,2025-06-17 00:53:59,0,"22,364,854","22,363,456","22,363,456","22,139,517"
1,202505,TOL,TB3R000601D1CG,TOL Inflow M1 Connected : Consumer - GEO Chann...,2025-06-17 00:53:59,0,"22,364,854","22,364,854","22,364,854","22,364,854"


In [26]:
''' TOL Connected : DataService (All Channel) '''

v_product_grp = 'TOL'

tol_conn_dsv_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_dsv_df = tol_conn_dsv_df.loc[tol_conn_dsv_df['METRIC_NAME'].str.contains('DataService')]
tol_conn_dsv_df = tol_conn_dsv_df.loc[~tol_conn_dsv_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_dsv_df = tol_conn_dsv_df.loc[tol_conn_dsv_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_dsv_df = tol_conn_dsv_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_dsv_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202505,TOL,TB3R000602CS,TOL Inflow M1 Connected : DataService,2025-06-17 00:53:59,"1,722,859","1,722,859","225,751","225,751","225,751"
1,202505,TOL,TB3R000602D1CS,TOL Inflow M1 Connected : DataService (Install...,2025-06-17 00:53:59,"1,722,859","1,722,859","225,751","225,751","225,751"


In [27]:
''' TOL Connected : DataService (Geo Channel) '''

v_product_grp = 'TOL'

tol_conn_dsv_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.loc[tol_conn_dsv_geo_df['METRIC_NAME'].str.contains('DataService')]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.loc[~tol_conn_dsv_geo_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.loc[tol_conn_dsv_geo_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_dsv_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202505,TOL,TB3R000602CS,TOL Inflow M1 Connected : DataService,2025-06-17 00:53:59,"1,722,859","1,722,859","225,751","225,751","225,751"
1,202505,TOL,TB3R000602D1CS,TOL Inflow M1 Connected : DataService (Install...,2025-06-17 00:53:59,"1,722,859","1,722,859","225,751","225,751","225,751"


### TVS

In [28]:
''' TVS Now '''

v_product_grp = 'TVS'

tvs_now_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tvs_now_df = tvs_now_df.loc[tvs_now_df['METRIC_NAME'].str.contains('Now')]
tvs_now_df = tvs_now_df.loc[tvs_now_df['TM_KEY_MTH']==v_tm_key_mth]
tvs_now_df = tvs_now_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tvs_now_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202505,TVS,TB4R001700CS,TVS Now Inflow M1,2025-06-17 00:53:59,"556,972","556,972","553,991","583,952","590,226"
1,202505,TVS,TB4R001700CG,TVS Now Inflow M1 - GEO Channel,2025-06-17 00:53:59,0,"556,079","553,880","583,841","590,115"


In [29]:
''' TVS CMDU '''

v_product_grp = 'TVS'

tvs_cmdu_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tvs_cmdu_df = tvs_cmdu_df.loc[tvs_cmdu_df['METRIC_NAME'].str.contains('CMDU')]
tvs_cmdu_df = tvs_cmdu_df.loc[tvs_cmdu_df['TM_KEY_MTH']==v_tm_key_mth]
tvs_cmdu_df = tvs_cmdu_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tvs_cmdu_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202505,TVS,TB4R001004CS,TVS CMDU Inflow M1,2025-06-17 00:53:59,"216,305","216,305","224,716","224,716","224,716"
1,202505,TVS,TB4R001004CG,TVS CMDU Inflow M1 - GEO Channel,2025-06-17 00:53:59,0,600,600,600,600


## ** Special Metric (All & Geo)

### Inflow (All Channel)
    B0R00010001CS   Total Inflow M1
    TB0R00010001CS  Total Inflow M1 : TRUE
    DB0R00010001CS  Total Inflow M1 : DTAC

In [30]:
''' B0R00010001CS : Total Inflow M1 '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('TB1R000900CS', 'DB1R000900CS', 'TB2R010500CS', 'TB2R020500CS', 'DB2R010500CS', 'DB2R020500CS', 'TB3R000601CS', 'TB3R000602CS', 'TB4R001700CS', 'TB4R001004CS')

v_metric_list = [
    'B0R00010001CS' #Total Inflow M1

    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC

    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC

    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService

    , 'TB4R001700CS' #TVS Now Inflow M1
    , 'TB4R001004CS' #TVS CMDU Inflow M1
    ]

total_m1_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_m1_df = total_m1_df.loc[total_m1_df['METRIC_CD'].isin(v_metric_list)]

total_m1_df['TOTAL'] = np.where(total_m1_df['METRIC_CD']=='B0R00010001CS', total_m1_df['P'], 0)

total_m1_df['PRE_T'] = np.where(total_m1_df['METRIC_CD']=='TB1R000900CS', total_m1_df['P'], 0)
total_m1_df['PRE_D'] = np.where(total_m1_df['METRIC_CD']=='DB1R000900CS', total_m1_df['P'], 0)

total_m1_df['POST_B2C_T'] = np.where(total_m1_df['METRIC_CD']=='TB2R010500CS', total_m1_df['P'], 0)
total_m1_df['POST_B2C_D'] = np.where(total_m1_df['METRIC_CD']=='DB2R010500CS', total_m1_df['P'], 0)
total_m1_df['POST_B2B_T'] = np.where(total_m1_df['METRIC_CD']=='TB2R020500CS', total_m1_df['P'], 0)
total_m1_df['POST_B2B_D'] = np.where(total_m1_df['METRIC_CD']=='DB2R020500CS', total_m1_df['P'], 0)

total_m1_df['TOL_CON'] = np.where(total_m1_df['METRIC_CD']=='TB3R000601CS', total_m1_df['P'], 0)
total_m1_df['TOL_DSV'] = np.where(total_m1_df['METRIC_CD']=='TB3R000602CS', total_m1_df['P'], 0)

total_m1_df['TVS_NOW'] = np.where(total_m1_df['METRIC_CD']=='TB4R001700CS', total_m1_df['P'], 0)
total_m1_df['TVS_CMDU'] = np.where(total_m1_df['METRIC_CD']=='TB4R001004CS', total_m1_df['P'], 0)

total_m1_df['CHK_SUM'] = np.where(total_m1_df['METRIC_CD'].isin(['TB1R000900CS', 'DB1R000900CS', 'TB2R010500CS', 'DB2R010500CS', 'TB2R020500CS', 'DB2R020500CS', 'TB3R000601CS', 'TB3R000602CS', 'TB4R001700CS', 'TB4R001004CS']), total_m1_df['P'], 0)

total_m1_df = total_m1_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CON':'sum', 'TOL_DSV':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum', 'CHK_SUM':'sum'})
total_m1_df['CHK_DIFF'] = total_m1_df['TOTAL'] - total_m1_df['CHK_SUM']
total_m1_df = total_m1_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_m1_df = total_m1_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL', 'PRE_T', 'PRE_D', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CON', 'TOL_DSV', 'TVS_NOW', 'TVS_CMDU']]

mod_col_list = total_m1_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_m1_df[col] = total_m1_df[col].apply(lambda x: format(x, ',.0f'))
total_m1_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL,PRE_T,PRE_D,POST_B2C_T,POST_B2C_D,POST_B2B_T,POST_B2B_D,TOL_CON,TOL_DSV,TVS_NOW,TVS_CMDU
0,202501,2025-06-17 00:53:59,-0,"639,956,235","639,956,235","225,749,189","283,042,228","61,334,089","18,126,426","4,946,013","18,864,945","25,646,982","829,033","382,335","1,034,995"
1,202502,2025-06-17 00:53:59,0,"575,180,401","575,180,401","192,368,192","249,735,487","63,185,463","17,947,144","4,637,783","19,426,521","25,046,745","1,648,415","267,183","917,468"
2,202503,2025-06-17 00:53:59,0,"563,517,561","563,517,561","201,578,035","226,924,550","70,909,630","17,637,272","5,801,792","8,532,545","28,791,201","2,517,766","470,671","354,099"
3,202504,2025-06-17 00:53:59,-0,"534,036,147","534,036,147","191,822,637","204,019,741","69,118,915","18,670,677","5,402,830","15,549,925","26,236,110","2,340,768","441,580","432,964"
4,202505,2025-06-17 00:53:59,0,"532,626,295","532,626,295","190,514,414","209,649,746","59,723,743","19,443,366","6,239,659","16,299,019","28,260,212","1,722,859","556,972","216,305"
5,202506,2025-06-17 00:53:59,0,"217,811,879","217,811,879","72,022,348","87,455,262","30,008,488","8,467,593","2,055,446","3,199,122","13,738,178","478,936","230,970","155,536"


In [31]:
''' TB0R00010001CS : Total Inflow M1 : TRUE '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('TB1R000900CS', 'TB2R010500CS', 'TB2R020500CS', 'TB3R000601CS', 'TB3R000602CS', 'TB4R001700CS', 'TB4R001004CS')

v_metric_list = [
    'TB0R00010001CS' #Total Inflow M1 : TRUE

    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH

    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH

    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService

    , 'TB4R001700CS' #TVS Now Inflow M1
    , 'TB4R001004CS' #TVS CMDU Inflow M1
    ]

total_m1_true_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_m1_true_df = total_m1_true_df.loc[total_m1_true_df['METRIC_CD'].isin(v_metric_list)]

total_m1_true_df['TOTAL_T'] = np.where(total_m1_true_df['METRIC_CD']=='TB0R00010001CS', total_m1_true_df['P'], 0)

total_m1_true_df['PRE_T'] = np.where(total_m1_true_df['METRIC_CD']=='TB1R000900CS', total_m1_true_df['P'], 0)

total_m1_true_df['POST_B2C_T'] = np.where(total_m1_true_df['METRIC_CD']=='TB2R010500CS', total_m1_true_df['P'], 0)
total_m1_true_df['POST_B2B_T'] = np.where(total_m1_true_df['METRIC_CD']=='TB2R020500CS', total_m1_true_df['P'], 0)

total_m1_true_df['TOL_CON'] = np.where(total_m1_true_df['METRIC_CD']=='TB3R000601CS', total_m1_true_df['P'], 0)
total_m1_true_df['TOL_DSV'] = np.where(total_m1_true_df['METRIC_CD']=='TB3R000602CS', total_m1_true_df['P'], 0)

total_m1_true_df['TVS_NOW'] = np.where(total_m1_true_df['METRIC_CD']=='TB4R001700CS', total_m1_true_df['P'], 0)
total_m1_true_df['TVS_CMDU'] = np.where(total_m1_true_df['METRIC_CD']=='TB4R001004CS', total_m1_true_df['P'], 0)

total_m1_true_df['CHK_SUM'] = np.where(total_m1_true_df['METRIC_CD'].isin(['TB1R000900CS', 'TB2R010500CS', 'TB2R020500CS', 'TB3R000601CS', 'TB3R000602CS', 'TB4R001700CS', 'TB4R001004CS']), total_m1_true_df['P'], 0)

total_m1_true_df = total_m1_true_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_T':'sum', 'PRE_T':'sum', 'POST_B2C_T':'sum', 'POST_B2B_T':'sum', 'TOL_CON':'sum', 'TOL_DSV':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum', 'CHK_SUM':'sum'})
total_m1_true_df['CHK_DIFF'] = total_m1_true_df['TOTAL_T'] - total_m1_true_df['CHK_SUM']
total_m1_true_df = total_m1_true_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_m1_true_df = total_m1_true_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_T', 'PRE_T', 'POST_B2C_T', 'POST_B2B_T', 'TOL_CON', 'TOL_DSV', 'TVS_NOW', 'TVS_CMDU']]

mod_col_list = total_m1_true_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_m1_true_df[col] = total_m1_true_df[col].apply(lambda x: format(x, ',.0f'))
total_m1_true_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_T,PRE_T,POST_B2C_T,POST_B2B_T,TOL_CON,TOL_DSV,TVS_NOW,TVS_CMDU
0,202501,2025-06-17 00:53:59,0,"319,922,636","319,922,636","225,749,189","61,334,089","4,946,013","25,646,982","829,033","382,335","1,034,995"
1,202502,2025-06-17 00:53:59,0,"288,071,249","288,071,249","192,368,192","63,185,463","4,637,783","25,046,745","1,648,415","267,183","917,468"
2,202503,2025-06-17 00:53:59,0,"310,423,195","310,423,195","201,578,035","70,909,630","5,801,792","28,791,201","2,517,766","470,671","354,099"
3,202504,2025-06-17 00:53:59,0,"295,795,804","295,795,804","191,822,637","69,118,915","5,402,830","26,236,110","2,340,768","441,580","432,964"
4,202505,2025-06-17 00:53:59,0,"287,234,164","287,234,164","190,514,414","59,723,743","6,239,659","28,260,212","1,722,859","556,972","216,305"
5,202506,2025-06-17 00:53:59,0,"118,689,901","118,689,901","72,022,348","30,008,488","2,055,446","13,738,178","478,936","230,970","155,536"


In [32]:
''' DB0R00010001CS : Total Inflow M1 : DTAC '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB1R000900CS', 'DB2R010500CS', 'DB2R020500CS')

v_metric_list = [
    'DB0R00010001CS' #Total Inflow M1 : DTAC

    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC
    
    , 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC
    ]

total_m1_dtac_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_m1_dtac_df = total_m1_dtac_df.loc[total_m1_dtac_df['METRIC_CD'].isin(v_metric_list)]

total_m1_dtac_df['TOTAL_D'] = np.where(total_m1_dtac_df['METRIC_CD']=='DB0R00010001CS', total_m1_dtac_df['P'], 0)

total_m1_dtac_df['PRE_D'] = np.where(total_m1_dtac_df['METRIC_CD']=='DB1R000900CS', total_m1_dtac_df['P'], 0)

total_m1_dtac_df['POST_B2C_D'] = np.where(total_m1_dtac_df['METRIC_CD']=='DB2R010500CS', total_m1_dtac_df['P'], 0)
total_m1_dtac_df['POST_B2B_D'] = np.where(total_m1_dtac_df['METRIC_CD']=='DB2R020500CS', total_m1_dtac_df['P'], 0)

total_m1_dtac_df['CHK_SUM'] = np.where(total_m1_dtac_df['METRIC_CD'].isin(['DB1R000900CS', 'DB2R010500CS', 'DB2R020500CS']), total_m1_dtac_df['P'], 0)

total_m1_dtac_df = total_m1_dtac_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_D':'sum', 'PRE_D':'sum', 'POST_B2C_D':'sum', 'POST_B2B_D':'sum', 'CHK_SUM':'sum'})
total_m1_dtac_df['CHK_DIFF'] = total_m1_dtac_df['TOTAL_D'] - total_m1_dtac_df['CHK_SUM']
total_m1_dtac_df = total_m1_dtac_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_m1_dtac_df = total_m1_dtac_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_D', 'PRE_D', 'POST_B2C_D', 'POST_B2B_D']]

mod_col_list = total_m1_dtac_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_m1_dtac_df[col] = total_m1_dtac_df[col].apply(lambda x: format(x, ',.0f'))
total_m1_dtac_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_D,PRE_D,POST_B2C_D,POST_B2B_D
0,202501,2025-06-17 00:53:59,0,"320,033,599","320,033,599","283,042,228","18,126,426","18,864,945"
1,202502,2025-06-17 00:53:59,0,"287,109,152","287,109,152","249,735,487","17,947,144","19,426,521"
2,202503,2025-06-17 00:53:59,0,"253,094,366","253,094,366","226,924,550","17,637,272","8,532,545"
3,202504,2025-06-17 00:53:59,0,"238,240,343","238,240,343","204,019,741","18,670,677","15,549,925"
4,202505,2025-06-17 00:53:59,0,"245,392,131","245,392,131","209,649,746","19,443,366","16,299,019"
5,202506,2025-06-17 00:53:59,0,"99,121,977","99,121,977","87,455,262","8,467,593","3,199,122"


### Inflow (Geo Channel)
    B0R00010001CG   Total Inflow M1 - GEO Channel
    TB0R00010001CG  Total Inflow M1 : TRUE - GEO Channel
    DB0R00010001CG  Total Inflow M1 : DTAC - GEO Channel

In [33]:
''' B0R00010001CG : Total Inflow M1 - GEO Channel '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('TB1R000900CG', 'DB1R000900CG', 'TB2R010500CG', 'DB2R010500CG', 'TB2R020500CG', 'DB2R020500CG', 'TB3R000601CG', 'TB3R000602CG', 'TB4R001700CG', 'TB4R001004CG')

v_metric_list = [
    'B0R00010001CG' #Total Inflow M1 - GEO Channel
    
    , 'TB1R000900CG' #Prepaid Inflow M1 : TMH - GEO Channel
    , 'DB1R000900CG' #Prepaid Inflow M1 : DTAC - GEO Channel
    
    , 'TB2R010500CG' #Postpaid Inflow M1 B2C : TMH - GEO Channel
    , 'DB2R010500CG' #Postpaid Inflow M1 B2C : DTAC - GEO Channel
    , 'TB2R020500CG' #Postpaid Inflow M1 B2B : TMH - GEO Channel
    , 'DB2R020500CG' #Postpaid Inflow M1 B2B : DTAC - GEO Channel
    
    , 'TB3R000601CG' #TOL Inflow M1 Connected : Consumer - GEO Channel
    , 'TB3R000602CG' #TOL Inflow M1 Connected : DataService - GEO Channel
    
    , 'TB4R001700CG' #TVS Now Inflow M1 - GEO Channel
    , 'TB4R001004CG' #TVS CMDU Inflow M1 - GEO Channel
    ]

total_m1_geo_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_m1_geo_df = total_m1_geo_df.loc[total_m1_geo_df['METRIC_CD'].isin(v_metric_list)]

total_m1_geo_df['TOTAL'] = np.where(total_m1_geo_df['METRIC_CD']=='B0R00010001CG', total_m1_geo_df['P'], 0)

total_m1_geo_df['PRE_T'] = np.where(total_m1_geo_df['METRIC_CD']=='TB1R000900CG', total_m1_geo_df['P'], 0)
total_m1_geo_df['PRE_D'] = np.where(total_m1_geo_df['METRIC_CD']=='DB1R000900CG', total_m1_geo_df['P'], 0)

total_m1_geo_df['POST_B2C_T'] = np.where(total_m1_geo_df['METRIC_CD']=='TB2R010500CG', total_m1_geo_df['P'], 0)
total_m1_geo_df['POST_B2C_D'] = np.where(total_m1_geo_df['METRIC_CD']=='DB2R010500CG', total_m1_geo_df['P'], 0)
total_m1_geo_df['POST_B2B_T'] = np.where(total_m1_geo_df['METRIC_CD']=='TB2R020500CG', total_m1_geo_df['P'], 0)
total_m1_geo_df['POST_B2B_D'] = np.where(total_m1_geo_df['METRIC_CD']=='DB2R020500CG', total_m1_geo_df['P'], 0)

total_m1_geo_df['TOL_CON'] = np.where(total_m1_geo_df['METRIC_CD']=='TB3R000601CG', total_m1_geo_df['P'], 0)
total_m1_geo_df['TOL_DSV'] = np.where(total_m1_geo_df['METRIC_CD']=='TB3R000602CG', total_m1_geo_df['P'], 0)

total_m1_geo_df['TVS_NOW'] = np.where(total_m1_geo_df['METRIC_CD']=='TB4R001700CG', total_m1_geo_df['P'], 0)
total_m1_geo_df['TVS_CMDU'] = np.where(total_m1_geo_df['METRIC_CD']=='TB4R001004CG', total_m1_geo_df['P'], 0)

total_m1_geo_df['CHK_SUM'] = np.where(total_m1_geo_df['METRIC_CD'].isin(['TB1R000900CG', 'DB1R000900CG', 'TB2R010500CG', 'DB2R010500CG', 'TB2R020500CG', 'DB2R020500CG', 'TB3R000601CG', 'TB3R000602CG', 'TB4R001700CG', 'TB4R001004CG']), total_m1_geo_df['P'], 0)

total_m1_geo_df = total_m1_geo_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CON':'sum', 'TOL_DSV':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum', 'CHK_SUM':'sum'})
total_m1_geo_df['CHK_DIFF'] = total_m1_geo_df['TOTAL'] - total_m1_geo_df['CHK_SUM']
total_m1_geo_df = total_m1_geo_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_m1_geo_df = total_m1_geo_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL', 'PRE_T', 'PRE_D', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CON', 'TOL_DSV', 'TVS_NOW', 'TVS_CMDU']]

mod_col_list = total_m1_geo_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_m1_geo_df[col] = total_m1_geo_df[col].apply(lambda x: format(x, ',.0f'))
total_m1_geo_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL,PRE_T,PRE_D,POST_B2C_T,POST_B2C_D,POST_B2B_T,POST_B2B_D,TOL_CON,TOL_DSV,TVS_NOW,TVS_CMDU
0,202501,2025-06-17 00:53:59,0,"560,918,001","560,918,001","223,753,635","275,241,571","31,477,331","7,077,828","2,357,448","381,005","19,471,134","397,930","376,664","383,456"
1,202502,2025-06-17 00:53:59,0,"494,465,664","494,465,664","190,565,264","240,052,088","32,971,912","7,014,205","2,468,889","382,012","19,961,102","423,582","264,749","361,861"
2,202503,2025-06-17 00:53:59,0,"495,388,907","495,388,907","199,692,427","222,125,906","38,509,836","7,839,038","3,206,656","392,224","22,964,967","133,660","470,449","53,744"
3,202504,2025-06-17 00:53:59,0,"458,678,603","458,678,603","189,643,908","199,028,432","37,483,652","7,699,989","2,886,311","387,725","20,901,734","134,885","441,246","70,723"
4,202505,2025-06-17 00:53:59,0,"459,410,712","459,410,712","188,609,826","207,177,802","29,453,492","7,516,555","3,110,301","399,840","22,364,854","221,363","556,079",600
5,202506,2025-06-17 00:53:59,0,"186,465,216","186,465,216","71,052,194","85,743,875","13,895,139","3,481,328","916,661","250,923","10,841,208","47,382","230,525","5,981"


In [34]:
''' TB0R00010001CG : Total Inflow M1 : TRUE - GEO Channel '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('TB1R000900CG', 'TB2R010500CG', 'TB2R020500CG', 'TB3R000601CG', 'TB3R000602CG', 'TB4R001700CG', 'TB4R001004CG')

v_metric_list = [
    'TB0R00010001CG' #Total Inflow M1 - GEO Channel
    
    , 'TB1R000900CG' #Prepaid Inflow M1 : TMH - GEO Channel
    
    , 'TB2R010500CG' #Postpaid Inflow M1 B2C : TMH - GEO Channel
    , 'TB2R020500CG' #Postpaid Inflow M1 B2B : TMH - GEO Channel
    
    , 'TB3R000601CG' #TOL Inflow M1 Connected : Consumer - GEO Channel
    , 'TB3R000602CG' #TOL Inflow M1 Connected : DataService - GEO Channel
    
    , 'TB4R001700CG' #TVS Now Inflow M1 - GEO Channel
    , 'TB4R001004CG' #TVS CMDU Inflow M1 - GEO Channel
    ]

total_m1_true_geo_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_m1_true_geo_df = total_m1_true_geo_df.loc[total_m1_true_geo_df['METRIC_CD'].isin(v_metric_list)]

total_m1_true_geo_df['TOTAL_T'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB0R00010001CG', total_m1_true_geo_df['P'], 0)

total_m1_true_geo_df['PRE_T'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB1R000900CG', total_m1_true_geo_df['P'], 0)

total_m1_true_geo_df['POST_B2C_T'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB2R010500CG', total_m1_true_geo_df['P'], 0)
total_m1_true_geo_df['POST_B2B_T'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB2R020500CG', total_m1_true_geo_df['P'], 0)

total_m1_true_geo_df['TOL_CON'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB3R000601CG', total_m1_true_geo_df['P'], 0)
total_m1_true_geo_df['TOL_DSV'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB3R000602CG', total_m1_true_geo_df['P'], 0)

total_m1_true_geo_df['TVS_NOW'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB4R001700CG', total_m1_true_geo_df['P'], 0)
total_m1_true_geo_df['TVS_CMDU'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB4R001004CG', total_m1_true_geo_df['P'], 0)

total_m1_true_geo_df['CHK_SUM'] = np.where(total_m1_true_geo_df['METRIC_CD'].isin(['TB1R000900CG', 'TB2R010500CG', 'TB2R020500CG', 'TB3R000601CG', 'TB3R000602CG', 'TB4R001700CG', 'TB4R001004CG']), total_m1_true_geo_df['P'], 0)

total_m1_true_geo_df = total_m1_true_geo_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_T':'sum', 'PRE_T':'sum', 'POST_B2C_T':'sum', 'POST_B2B_T':'sum', 'TOL_CON':'sum', 'TOL_DSV':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum', 'CHK_SUM':'sum'})
total_m1_true_geo_df['CHK_DIFF'] = total_m1_true_geo_df['TOTAL_T'] - total_m1_true_geo_df['CHK_SUM']
total_m1_true_geo_df = total_m1_true_geo_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_m1_true_geo_df = total_m1_true_geo_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_T', 'PRE_T', 'POST_B2C_T', 'POST_B2B_T', 'TOL_CON', 'TOL_DSV', 'TVS_NOW', 'TVS_CMDU']]

mod_col_list = total_m1_true_geo_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_m1_true_geo_df[col] = total_m1_true_geo_df[col].apply(lambda x: format(x, ',.0f'))
total_m1_true_geo_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_T,PRE_T,POST_B2C_T,POST_B2B_T,TOL_CON,TOL_DSV,TVS_NOW,TVS_CMDU
0,202501,2025-06-17 00:53:59,0,"278,217,598","278,217,598","223,753,635","31,477,331","2,357,448","19,471,134","397,930","376,664","383,456"
1,202502,2025-06-17 00:53:59,0,"247,017,359","247,017,359","190,565,264","32,971,912","2,468,889","19,961,102","423,582","264,749","361,861"
2,202503,2025-06-17 00:53:59,0,"265,031,740","265,031,740","199,692,427","38,509,836","3,206,656","22,964,967","133,660","470,449","53,744"
3,202504,2025-06-17 00:53:59,0,"251,562,458","251,562,458","189,643,908","37,483,652","2,886,311","20,901,734","134,885","441,246","70,723"
4,202505,2025-06-17 00:53:59,0,"244,316,516","244,316,516","188,609,826","29,453,492","3,110,301","22,364,854","221,363","556,079",600
5,202506,2025-06-17 00:53:59,0,"96,989,090","96,989,090","71,052,194","13,895,139","916,661","10,841,208","47,382","230,525","5,981"


In [35]:
''' DB0R00010001CG : Total Inflow M1 : DTAC - GEO Channel '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB1R000900CG', 'DB2R010500CG', 'DB2R020500CG')

v_metric_list = [
    'DB0R00010001CG' #Total Inflow M1 : DTAC - GEO Channel
    
    , 'DB1R000900CG' #Prepaid Inflow M1 : DTAC - GEO Channel
    
    , 'DB2R010500CG' #Postpaid Inflow M1 B2C : DTAC - GEO Channel
    , 'DB2R020500CG' #Postpaid Inflow M1 B2B : DTAC - GEO Channel
    ]

total_m1_dtac_geo_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_m1_dtac_geo_df = total_m1_dtac_geo_df.loc[total_m1_dtac_geo_df['METRIC_CD'].isin(v_metric_list)]

total_m1_dtac_geo_df['TOTAL_D'] = np.where(total_m1_dtac_geo_df['METRIC_CD']=='DB0R00010001CG', total_m1_dtac_geo_df['P'], 0)

total_m1_dtac_geo_df['PRE_D'] = np.where(total_m1_dtac_geo_df['METRIC_CD']=='DB1R000900CG', total_m1_dtac_geo_df['P'], 0)

total_m1_dtac_geo_df['POST_B2C_D'] = np.where(total_m1_dtac_geo_df['METRIC_CD']=='DB2R010500CG', total_m1_dtac_geo_df['P'], 0)
total_m1_dtac_geo_df['POST_B2B_D'] = np.where(total_m1_dtac_geo_df['METRIC_CD']=='DB2R020500CG', total_m1_dtac_geo_df['P'], 0)

total_m1_dtac_geo_df['CHK_SUM'] = np.where(total_m1_dtac_geo_df['METRIC_CD'].isin(['DB1R000900CG', 'DB2R010500CG', 'DB2R020500CG']), total_m1_dtac_geo_df['P'], 0)

total_m1_dtac_geo_df = total_m1_dtac_geo_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_D':'sum', 'PRE_D':'sum', 'POST_B2C_D':'sum', 'POST_B2B_D':'sum', 'CHK_SUM':'sum'})
total_m1_dtac_geo_df['CHK_DIFF'] = total_m1_dtac_geo_df['TOTAL_D'] - total_m1_dtac_geo_df['CHK_SUM']
total_m1_dtac_geo_df = total_m1_dtac_geo_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_m1_dtac_geo_df = total_m1_dtac_geo_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_D', 'PRE_D', 'POST_B2C_D', 'POST_B2B_D']]

mod_col_list = total_m1_dtac_geo_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_m1_dtac_geo_df[col] = total_m1_dtac_geo_df[col].apply(lambda x: format(x, ',.0f'))
total_m1_dtac_geo_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_D,PRE_D,POST_B2C_D,POST_B2B_D
0,202501,2025-06-17 00:53:59,-0,"282,700,403","282,700,403","275,241,571","7,077,828","381,005"
1,202502,2025-06-17 00:53:59,0,"247,448,304","247,448,304","240,052,088","7,014,205","382,012"
2,202503,2025-06-17 00:53:59,0,"230,357,167","230,357,167","222,125,906","7,839,038","392,224"
3,202504,2025-06-17 00:53:59,-0,"207,116,145","207,116,145","199,028,432","7,699,989","387,725"
4,202505,2025-06-17 00:53:59,0,"215,094,197","215,094,197","207,177,802","7,516,555","399,840"
5,202506,2025-06-17 00:53:59,0,"89,476,126","89,476,126","85,743,875","3,481,328","250,923"
